<a href="https://colab.research.google.com/github/milvus-io/bootcamp/blob/master/bootcamp/RAG/advanced_rag/sentence_window_with_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sentence window

## Google Colab preparation[optional]
This is an optional step, if you want to run this notebook on Google Colab.

In [ ]:
! git clone https://github.com/milvus-io/bootcamp.git

In [ ]:
import shutil
src_dir = "./bootcamp/bootcamp/RAG/advanced_rag/rag_utils"
dst_dir = "./rag_utils"
shutil.copytree(src_dir, dst_dir)

In [ ]:
! pip install --upgrade langchain langchain-community langchain_milvus langchain-openai bs4

Please prepare you [OPENAI_API_KEY](https://openai.com/index/openai-api/) in your environment variables.
![](imgs/colab_api_key1.png)

### If you are running this notebook on Google Colab, you have to restart this session by `Cmd/Ctrl + M`, then press `.` to make the environment take effect.

In [ ]:
from google.colab import userdata
import os

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

----
## Get started
![](imgs/sentence_window.png)

## Prepare the data

We use the Langchain WebBaseLoader to load documents from [blog sources](https://lilianweng.github.io/posts/2023-06-23-agent/) and split them into chunks using the RecursiveCharacterTextSplitter.

In [1]:
import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

from rag_utils.vanilla import vectorstore

# Create a WebBaseLoader instance to load documents from web sources
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
# Load documents from web sources using the loader
documents = loader.load()
# Initialize a RecursiveCharacterTextSplitter for splitting text into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

# Split the documents into chunks using the text_splitter
docs = text_splitter.split_documents(documents)

We add the wider window text information to each chunk using the `write_wider_window` function. We can see the metadata of the document contains a `wider_text` field, which is the wider window text information.

In [2]:
from rag_utils.sentence_window import write_wider_window

write_wider_window(docs, documents[0], offset=500)

docs[1], len(docs)

(Document(page_content='Short-term memory: I would consider all the in-context learning (See Prompt Engineering) as utilizing short-term memory of the model to learn.\nLong-term memory: This provides the agent with the capability to retain and recall (infinite) information over extended periods, often by leveraging an external vector store and fast retrieval.\n\n\nTool use\n\nThe agent learns to call external APIs for extra information that is missing from the model weights (often hard to change after pre-training), including current information, code execution capability, access to proprietary information sources and more.', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'start_index': 979, 'end_index': 1579, 'wider_text': 't System Overview#\nIn a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:\n\nPlanning\n\nSubgoal and decomposition: The agent breaks down large tasks into smaller, manageab

## Build the chain

We load the docs into milvus vectorstore, and build a milvus retriever.

In [3]:
vectorstore.add_documents(docs)
retriever = vectorstore.as_retriever()

Define the vanilla RAG chain.

In [4]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from rag_utils.vanilla import format_docs, rag_prompt, llm

vanilla_rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | rag_prompt
    | llm
    | StrOutputParser()
)

Define the sentence window chain.

In [5]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from rag_utils.sentence_window import format_docs_with_wider_window

sentence_window_chain = (
    {
        "context": retriever | format_docs_with_wider_window,
        "question": RunnablePassthrough(),
    }
    | rag_prompt
    | llm
    | StrOutputParser()
)

## Test the chain


In [6]:
query = "what are the different types of ANN algorithms?"

vanilla_result = vanilla_rag_chain.invoke(query)
sentence_window_result = sentence_window_chain.invoke(query)
print(
    f"[vanilla_result]:\n{vanilla_result}\n\n[sentence_window_result]:\n{sentence_window_result}"
)

[vanilla_result]:
The context provided does not list different types of Artificial Neural Network (ANN) algorithms. However, it does mention different algorithms used for Approximate Nearest Neighbors (ANN) search, which are LSH (Locality-Sensitive Hashing), ANNOY (Approximate Nearest Neighbors Oh Yeah), FAISS (Facebook AI Similarity Search), and ScaNN (Scalable Nearest Neighbors).

[sentence_window_result]:
The different types of Approximate Nearest Neighbors (ANN) algorithms mentioned in the context are:

1. LSH (Locality-Sensitive Hashing): This algorithm uses a hashing function to map similar input items to the same buckets with high probability.

2. ANNOY (Approximate Nearest Neighbors Oh Yeah): This algorithm uses random projection trees, a set of binary trees where each non-leaf node represents a hyperplane splitting the input space into half and each leaf stores one data point.

3. HNSW (Hierarchical Navigable Small World): This algorithm is inspired by the idea of small world 

We can see that the sentence window results include `HNSW`, but the vanilla results do not. This is because the context window of the sentence window is wider, so it can contain more relevant content fragments to reduce the loss of information.